In [11]:
!pip install biopython

from Bio import SeqIO
from Bio.Blast import NCBIXML
from Bio.Blast.Applications import NcbiblastnCommandline
import os

# 定义输入文件和输出文件路径
human_file = "human.fa"
mouse_file = "mouse.fa"
output_file = "blast_results.txt"

# 设置BLAST搜索参数
blast_exe = "blastn"  # 'blastn'用于核酸序列比对，如果是蛋白质序列，请改为'blastp'

# 确保BLAST数据库已构建
if not os.path.exists("mouse"):
    os.system(f"makeblastdb -in {mouse_file} -dbtype nucl -out mouse")

# 打开输出文件
with open(output_file, "w") as out_handle:
    for record in SeqIO.parse(human_file, "fasta"):
        # 保存人类序列ID
        human_id = record.id
        
        # 写入临时文件以便使用BLAST
        with open("temp_human.fasta", "w") as temp_query:
            SeqIO.write(record, temp_query, "fasta")

        # 设置BLAST搜索命令
        blastn_cline = NcbiblastnCommandline(
            cmd=blast_exe,
            query="temp_human.fasta",
            db="mouse",
            evalue=0.001,
            outfmt=5,
            out="temp_blast.xml"
        )

        # 运行BLAST命令
        stdout, stderr = blastn_cline()
        
        # 解析BLAST XML结果
        with open("temp_blast.xml") as result_handle:
            blast_record = NCBIXML.read(result_handle)
            if blast_record.alignments:
                # 获取最佳匹配结果
                best_alignment = blast_record.alignments[0]
                hsp = best_alignment.hsps[0]
                
                # 写入输出文件
                out_handle.write(f"Human ID: {human_id}\n")
                out_handle.write(f"Mouse ID: {best_alignment.hit_id}\n")
                out_handle.write("Alignment:\n")
                out_handle.write(f"{hsp.query}\n")
                out_handle.write(f"{hsp.match}\n")
                out_handle.write(f"{hsp.sbjct}\n")
                out_handle.write(f"E-value: {hsp.expect}\n")
                out_handle.write(f"Bit Score: {hsp.bits}\n")
                out_handle.write("\n" + "="*50 + "\n\n")

# 清理临时文件
os.remove("temp_human.fasta")
os.remove("temp_blast.xml")


#Furthermore question
#1.blastn is chosen for nucleotide sequence alignment, as human.fa and mouse.fa likely contain nucleotide sequences.
#2. The standard nucleotide scoring matrix is used for blastn; if using blastp for protein alignment, BLOSUM62 is typically selected.
#3.The E-value threshold is set to 0.001 to ensure that only highly significant matches are reported.

ApplicationError: Non-zero return code 1 from 'blastn -out temp_blast.xml -outfmt 5 -query temp_human.fasta -db mouse -evalue 0.001', message "'blastn' is not recognized as an internal or external command,"